# $t$-test

In [46]:
import polars as pl
import numpy as np
from scipy import stats as sps

## 獨立樣本

In [47]:
rvs1 = sps.norm.rvs(loc=5, scale=10, size=500, random_state=1)
rvs2 = sps.norm.rvs(loc=5, scale=20, size=500, random_state=1)

assert isinstance(rvs1, np.ndarray)
assert isinstance(rvs2, np.ndarray)

首先進行 levene 分析。

In [48]:
levene = sps.levene(rvs1, rvs2)

levene

LeveneResult(statistic=np.float64(176.01040622090554), pvalue=np.float64(4.090185632954182e-37))

如果 $p_{\text{levene}}$ 落在拒絕域（也就是 $\leq 0.05$），則使用 Welch's $t$-test；否則使用 Student's $t$-test。

In [49]:
print("Should we reject H0?", levene.pvalue <= 0.05)

Should we reject H0? True


In [50]:
ttest = sps.ttest_ind(rvs1, rvs2, equal_var=levene.pvalue > 0.05)

ttest

TtestResult(statistic=np.float64(-0.540094946754036), pvalue=np.float64(0.5892954193870468), df=np.float64(733.8235294117648))

### 從 Excel 讀取資料

In [51]:
df = pl.read_excel('grades.xlsx', sheet_name='Sheet1')
df

f1,f2
i64,i64
85,82
82,75
90,80
90,80
75,85
88,85
87,75
85,80
78,70


In [52]:
levene = sps.levene(df["f1"], df["f2"])
print("Should we reject H0?", levene.pvalue <= 0.05)

Should we reject H0? False


In [53]:
ttest = sps.ttest_ind(rvs1, rvs2, equal_var=levene.pvalue > 0.05)

ttest

TtestResult(statistic=np.float64(-0.540094946754036), pvalue=np.float64(0.5892520506239134), df=np.float64(998.0))

In [54]:
print("甲乙兩班的成績，有顯著差異嗎？", ttest.pvalue <= 0.05)  # type: ignore

甲乙兩班的成績，有顯著差異嗎？ False


## 相依樣本

In [55]:
mid = [78, 80, 90, 90, 70, 88, 82, 74, 65, 85]
final = [84, 83, 89, 90, 78, 89, 87, 84, 78, 80]

In [56]:
ttest = sps.ttest_rel(mid, final)
ttest

TtestResult(statistic=np.float64(-2.3094010767585034), pvalue=np.float64(0.04628184731527353), df=np.int64(9))

### 從 Excel 讀取資料

In [57]:
df = pl.read_excel('eqgrade.xlsx')
df

husband,wife
i64,i64
34,38
22,39
38,35
20,25
29,19
…,…
14,44
23,40
40,45


In [58]:
ttest = sps.ttest_rel(df["husband"], df["wife"])
ttest


TtestResult(statistic=np.float64(-1.9977374136430808), pvalue=np.float64(0.07107892367821363), df=np.int64(11))

In [59]:
print("丈夫與妻子之間的 EQ 是否有顯著差異？", ttest.pvalue <= 0.05)  # type: ignore

丈夫與妻子之間的 EQ 是否有顯著差異？ False
